In [1]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
#read the data set 
dataset=pd.read_csv("Social_Network_Ads.csv")
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [3]:
#converting categorical data to numerical data
dataset=pd.get_dummies(dataset,drop_first=True)
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [4]:
#Removing the unwanted cloumns from table
dataset=dataset.drop("User ID",axis=1)
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [5]:
##input and output Split form data 
indep=dataset[['Age', 'EstimatedSalary','Gender_Male']]
dep=dataset['Purchased']

In [6]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [7]:
#standardize the value using StandardScaler
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
param_grid = {'kernel':['linear','rbf','poly','sigmoid'],'decision_function_shape':['ovo','ovr'],
             'gamma':['auto','scale'],
             'C':[10.0,100.0,1000.0,2000.0,3000.0]} 
grid = GridSearchCV(SVC(probability=True), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted',) #when the Auc_cur_score if probability is false then attribute error shows then probability true means working
# fitting the model for grid search 
grid.fit(X_train,y_train) 

Fitting 5 folds for each of 80 candidates, totalling 400 fits


GridSearchCV(estimator=SVC(probability=True), n_jobs=-1,
             param_grid={'C': [10.0, 100.0, 1000.0, 2000.0, 3000.0],
                         'decision_function_shape': ['ovo', 'ovr'],
                         'gamma': ['auto', 'scale'],
                         'kernel': ['linear', 'rbf', 'poly', 'sigmoid']},
             scoring='f1_weighted', verbose=3)

In [9]:
# print best parameter after tuning 
re=grid.cv_results_
grid_predictions = grid.predict(X_test) 
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)
# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)
print('the confusion metrix is \n',cm)
print('the classification report is \n',clf_report)

the confusion metrix is 
 [[80  5]
 [ 7 42]]
the classification report is 
               precision    recall  f1-score   support

           0       0.92      0.94      0.93        85
           1       0.89      0.86      0.88        49

    accuracy                           0.91       134
   macro avg       0.91      0.90      0.90       134
weighted avg       0.91      0.91      0.91       134



In [10]:
#Calculate the F1Score 
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'C': 100.0, 'decision_function_shape': 'ovo', 'gamma': 'auto', 'kernel': 'rbf'}: 0.9100355779243318


In [11]:
#calculate the roc_auc_score 
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.9539015606242497

In [12]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_decision_function_shape,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.013022,0.007237,0.004059,0.005609,10.0,ovo,auto,linear,"{'C': 10.0, 'decision_function_shape': 'ovo', ...",0.776290,0.790949,0.698235,0.923510,0.901744,0.818146,0.083619,41
1,0.014382,0.002204,0.001116,0.001586,10.0,ovo,auto,rbf,"{'C': 10.0, 'decision_function_shape': 'ovo', ...",0.867478,0.886792,0.869709,0.944161,0.943041,0.902236,0.034431,5
2,0.011558,0.005937,0.001112,0.001379,10.0,ovo,auto,poly,"{'C': 10.0, 'decision_function_shape': 'ovo', ...",0.799620,0.808392,0.787943,0.924528,0.901744,0.844445,0.056918,21
3,0.006776,0.006876,0.002404,0.004808,10.0,ovo,auto,sigmoid,"{'C': 10.0, 'decision_function_shape': 'ovo', ...",0.762677,0.738916,0.655795,0.796284,0.766556,0.744045,0.047743,77
4,0.009386,0.007664,0.006332,0.007756,10.0,ovo,scale,linear,"{'C': 10.0, 'decision_function_shape': 'ovo', ...",0.776290,0.790949,0.698235,0.923510,0.901744,0.818146,0.083619,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0.038848,0.034918,0.004837,0.003997,3000.0,ovr,auto,sigmoid,"{'C': 3000.0, 'decision_function_shape': 'ovr'...",0.791752,0.714931,0.698113,0.796284,0.766556,0.753527,0.040048,71
76,6.179154,1.341748,0.001743,0.002029,3000.0,ovr,scale,linear,"{'C': 3000.0, 'decision_function_shape': 'ovr'...",0.776290,0.790949,0.698235,0.923510,0.901744,0.818146,0.083619,41
77,0.624785,0.229401,0.003126,0.004260,3000.0,ovr,scale,rbf,"{'C': 3000.0, 'decision_function_shape': 'ovr'...",0.826263,0.866968,0.851527,0.847020,0.883278,0.855011,0.019206,17
78,5.189904,0.316579,0.000377,0.000753,3000.0,ovr,scale,poly,"{'C': 3000.0, 'decision_function_shape': 'ovr'...",0.822092,0.765553,0.766556,0.924528,0.901744,0.836095,0.066541,29


In [13]:
Age_input=float(input("Age:"))
EstimatedSalary_input=float(input("EstimatedSalary:"))
Gender_input=float(input("Gender:"))


Age:25
EstimatedSalary:15000
Gender:1


In [16]:
Purchase_Prediction=grid.predict([[Age_input,EstimatedSalary_input,Gender_input]])
print(Purchase_Prediction)

[1]
